In [1]:
import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

In [2]:
list_series_error, df_input = [], pd.read_excel('./file/1.page.xlsx',
                                                header=0,
                                                dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)
df_input['SKU Count'] = df_input['SKU Count'].astype(int)
df_input['Page'] = df_input['Page'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：1



,No,Url,SKU Count,Page
0,1,https://www.dormanproducts.com/gsearch.aspx?ty...,93,1


In [3]:
(crawler_index, crawler_series) = next(df_input.iterrows())

crawler_index, crawler_series

(0,
 No                                                           1
 Url          https://www.dormanproducts.com/gsearch.aspx?ty...
 SKU Count                                                   93
 Page                                                         1
 Name: 0, dtype: object)

In [4]:
import time

count_cycle, list_url = 0, []
while len(list_url) < crawler_series['SKU Count']:
    count_cycle += 1

    for page in range(crawler_series['Page']):

        count_retry = 0
        while True:
            count_retry += 1
            print(count_cycle, page + 1, count_retry)
            try:
                url_request = f'''{crawler_series['Url']}&num=100&start={page * 100}'''

                resp = requests.get(url_request,
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(5, 15))

                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    list_href = html.xpath('//h2[@class="item-headline"]/a/@href')
                    if list_href:
                        if count_cycle == 1 and page == 0:
                            with open('./url.html', 'w', encoding='UTF-8') as file:
                                file.write(str(soup))
                        break
            except KeyboardInterrupt:
                break
            except:
                time.sleep(3)
                continue

        # = = = = = = = = = = = = = = =

        list_url.extend([f'https://www.dormanproducts.com/{href.strip()}' for href in list_href])
        list_url = list(dict.fromkeys(list_url))

        # = = = = = = = = = = = = = = =

        print(f' - {len(list_url)}/{crawler_series['SKU Count']}')

        # = = = = = = = = = = = = = = =

        if len(list_url) == crawler_series['SKU Count']:
            break

list_url

1 1 1
 - 93/93


['https://www.dormanproducts.com/p-17668-34198.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-7395-34197.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-6889-34187.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-25955-34188.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-32262-34189.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-89000-34275.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-87045-34257.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-86526-34258.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-84645-34248.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-85999-34272.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-85708-34274.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-19729-34195.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-7394-34192.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-6943-34202.aspx?origin=keyword',
 'https://www.dormanproducts.com/p-501

In [5]:
df_correct = pd.DataFrame({'No': range(1, len(list_url) + 1),
                           'Url': list_url})
df_correct.to_excel('./test_url.xlsx', index=False)

df_correct

,No,Url
0,1,https://www.dormanproducts.com/p-17668-34198.a...
1,2,https://www.dormanproducts.com/p-7395-34197.as...
2,3,https://www.dormanproducts.com/p-6889-34187.as...
3,4,https://www.dormanproducts.com/p-25955-34188.a...
4,5,https://www.dormanproducts.com/p-32262-34189.a...
...,...,...
88,89,https://www.dormanproducts.com/p-131747-34869....
89,90,https://www.dormanproducts.com/p-131983-34868....
90,91,https://www.dormanproducts.com/p-88995-34283.a...
91,92,https://www.dormanproducts.com/p-81798-419-500...


In [6]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://www.dormanproducts.com/gsearch.aspx?ty...
 SKU Count                                                     93
 Page                                                           1
 Request_Url    https://www.dormanproducts.com/gsearch.aspx?ty...
 Name: 0, dtype: object]

In [7]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_url_error.xlsx', index=False)

df_error

,No,Url,SKU Count,Page,Request_Url
0,1,https://www.dormanproducts.com/gsearch.aspx?ty...,93,1,https://www.dormanproducts.com/gsearch.aspx?ty...
